# Categorize Paintings based on individual emotional reactions
created by Sydney

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
import numpy as np

In [2]:
# Set which subject

subNum = 'pilot01'

In [3]:
# Read file

filename = "ratingData-{subNum}.csv".format(subNum=subNum)
df = pd.read_csv(filename)

FileNotFoundError: [Errno 2] File b'ratingData-pilot01.csv' does not exist: b'ratingData-pilot01.csv'

In [4]:
# Rename columns

df.rename(columns={"Unique ID":"ID",'a. Moved':'Moved','b. Fascinated':'Fascin','c. Funny':'Funny',
                   'd. Surprised':'Surpri','e. Indifferent':'Indiff','f. Calm':'Calm',
                   'g. Unsettling':'Unset','h. Beautiful':'Beaut'}, inplace=True)

In [50]:
# ORIGINAL GROUPINGS
    # Where Arousal is calculated with the sum of Moved, Fascinated, Funny, and Surprised
    # and Valence is calculated by the absolutely value of Unsettled - Indifferent, where a 
    # low Valence is Positive and a high Valence is negative. 
    
# df['Arousal'] = df['a. Moved']+df['b. Fascinated']+df['c. Funny']+df['d. Surprised']
# df['Valence'] = abs(df.loc[:,('g. Unsettling')] - df.loc[:,('e. Indifferent')])

In [51]:
# MODIFIED GROUPINGS 
    # Where Arousal is calculated with the sum of Moved, Fascinated, Funny, and Surprised
    # and Valence is equal to Unsettled, and Indifferent is the paintings that had the highest
    # indifference ratings

# First, take out the top 10 most indifferent paintings 
    # NOTE: change so that the values of the other categories matter? ie high indiff RELATIVE to
    # low scores in other categories?
df_indiff = df.sort_values(by=['Indiff'], ascending=False)
df_indiff = df_indiff.iloc[:10,:]

In [52]:
# Next, drop those paintings from main df

df = df.drop(df_indiff.index) # <--- ONLY DO THIS LINE IF MAKING AN INDIFFERENCE VARIABLE 
                              # (should be all after pilot01)

In [5]:
# Create a column of Arousal and Valence scores  

df['Arousal'] = df['Moved']+df['Fascin']+df['Funny']+df['Surpri']
df['Valence'] = df.loc[:,('Unset')]

In [6]:
# Section out groups

# get middle value
med = int(len(df)/2)

# create dfs that separate high arousal and low arousal
hA = df.iloc[0:med,:]
lA = df.iloc[med:len(df),:]

# sort those by Valuence scores
hA = hA.sort_values(by=['Valence'], ascending=False)
lA = lA.sort_values(by=['Valence'], ascending=False)

##### POSITIVE is if Valence score is low, NEGATIVE is if Valence score is high #####

hA_pos = hA.iloc[int(med/2):len(hA),:]
hA_neg = hA.iloc[0:int(med/2),:]

lA_pos = lA.iloc[int(med/2):len(lA),:]
lA_neg = lA.iloc[0:int(med/2),:]

# insert a col in each df with the group label
hA_pos.insert(4, 'Group', 'hiPos')
hA_neg.insert(4, 'Group', 'hiNeg')
lA_pos.insert(4, 'Group', 'loPos')
lA_neg.insert(4, 'Group', 'loNeg')

# df_indiff.insert(4, 'Group','indiff')

In [7]:
# Concat all groups into one df to output

outdf = pd.concat([hA_pos,hA_neg,lA_pos,lA_neg], sort=False)#, df_indiff], sort=False) 
outdf = outdf.drop(['Arousal','Valence'], axis=1) # dropping these bc in 'indiff', they are NaN. Also not helpful

In [8]:
# Export csv with everything

outdf.to_csv('{sub}_groups_w_ratings.csv'.format(sub=subNum))

### Extras for analysis: 

In [21]:
# Plot Histograms of scores

# fig, axes = plt.subplots(nrows=3, ncols=3, sharey = True, sharex = True, figsize=(12,10))

# emotionList=["Moved","Fascin","Funny","Surpri","Indiff","Calm","Unset","Beaut"]
# fig.suptitle("Num Ratings per Emotion: {sub}".format(sub=subNum), size=16)

# dim=331
# for i in range(1,9):
#     plt.subplot(dim+(i-1), \
#                       xlabel="Score", ylabel="Num Times Used",\
#                       xticks=np.arange(1,8,1)
#                      )
#     plt.hist(df.iloc[:,i+3], range=[1, 7], bins=7, align='left')
#     plt.title(emotionList[i-1])
                  
# plt.tight_layout(pad=3.0)
# fig.subplots_adjust(top=0.88)

In [23]:
# Correlation Matrix

# print("{sub} Correlation Matrix".format(sub=subNum))
# df.iloc[:,4:(len(df.columns)-2)].corr()

In [24]:
# Indexing examples: 

# get row of scores when given title
# df[df['Title']=='Torrey Pines']

# get rows where indifference score is greater than 4
# df[df['Indiff']>4]